In [ ]:
import gym
import gym.spaces
import enum
import numpy as np
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import ptan
import torch.optim as optim
import os
import csv
import glob
import numpy as np
import collections
from torch.optim import Adam
from d3rlpy.algos import DQN
from sklearn.model_selection import train_test_split
from d3rlpy.models.optimizers import OptimizerFactory
from d3rlpy.online.buffers import ReplayBuffer
from d3rlpy.online.explorers import LinearDecayEpsilonGreedy, ConstantEpsilonGreedy
from gym.utils import seeding
from gym.envs.registration import EnvSpec

Define some functions to prepare and load stocks data from csv files

In [ ]:



Prices = collections.namedtuple('Prices', field_names=['open', 'high', 'low', 'close', 'volume'])


def read_csv(file_name, sep=',', filter_data=True, fix_open_price=False):
    print("Reading", file_name)
    with open(file_name, 'rt', encoding='utf-8') as fd:
        reader = csv.reader(fd, delimiter=sep)
        h = next(reader)
        if '<OPEN>' not in h and sep == ',':
            return read_csv(file_name, ';')
        indices = [h.index(s) for s in ('<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>')]
        o, h, l, c, v = [], [], [], [], []
        count_out = 0
        count_filter = 0
        count_fixed = 0
        prev_vals = None
        for row in reader:
            vals = list(map(float, [row[idx] for idx in indices]))
            if filter_data and all(map(lambda v: abs(v-vals[0]) < 1e-8, vals[:-1])):
                count_filter += 1
                continue

            po, ph, pl, pc, pv = vals

            # fix open price for current bar to match close price for the previous bar
            if fix_open_price and prev_vals is not None:
                ppo, pph, ppl, ppc, ppv = prev_vals
                if abs(po - ppc) > 1e-8:
                    count_fixed += 1
                    po = ppc
                    pl = min(pl, po)
                    ph = max(ph, po)
            count_out += 1
            o.append(po)
            c.append(pc)
            h.append(ph)
            l.append(pl)
            v.append(pv)
            prev_vals = vals
    print("Read done, got %d rows, %d filtered, %d open prices adjusted" % (
        count_filter + count_out, count_filter, count_fixed))
    return Prices(open=np.array(o, dtype=np.float32),
                  high=np.array(h, dtype=np.float32),
                  low=np.array(l, dtype=np.float32),
                  close=np.array(c, dtype=np.float32),
                  volume=np.array(v, dtype=np.float32))


def prices_to_relative(prices):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open
    return Prices(open=prices.open, high=rh, low=rl, close=rc, volume=prices.volume)


def load_relative(csv_file):
    return prices_to_relative(read_csv(csv_file))


def price_files(dir_name):
    result = []
    for path in glob.glob(os.path.join(dir_name, "*.csv")):
        result.append(path)
    return result


def load_year_data(year, basedir='data'):
    y = str(year)[-2:]
    result = {}
    for path in glob.glob(os.path.join(basedir, "*_%s*.csv" % y)):
        result[path] = load_relative(path)
    return result

We encode all available actions as an enumerator's fields. We support a very simple
set of actions with only three options: do nothing, buy a single share, and close the
existing position

In [ ]:
DEFAULT_BARS_COUNT = 10
DEFAULT_COMMISSION_PERC = 0.1

class Actions(enum.Enum):
    Skip = 0
    Buy = 1
    Close = 2

State class, is a python class which implements most of the environment's functionality like
method calculates the current bar's close price. Prices passed to the State
class have the relative form in respect of the open price: the high, low, and close
components are relative ratios to the open price

In [ ]:
class State:
    def __init__(
        self,
        bars_count,
        commission_perc,
        reset_on_close,
        reward_on_close=True,
        volumes=True,
    ):
        assert isinstance(bars_count, int)
        assert bars_count > 0
        assert isinstance(commission_perc, float)
        assert commission_perc >= 0.0
        assert isinstance(reset_on_close, bool)
        assert isinstance(reward_on_close, bool)
        self.bars_count = bars_count
        self.commission_perc = commission_perc
        self.reset_on_close = reset_on_close
        self.reward_on_close = reward_on_close
        self.volumes = volumes

    def reset(self, prices, offset):
        assert isinstance(prices, Prices)
        assert offset >= self.bars_count - 1
        self.have_position = False
        self.open_price = 0.0
        self._prices = prices
        self._offset = offset

    @property
    def shape(self):
        # [h, l, c] * bars + position_flag + rel_profit
        if self.volumes:
            return (4 * self.bars_count + 1 + 1,)
        else:
            return (3 * self.bars_count + 1 + 1,)

    def encode(self):
        """
        Convert current state into numpy array.
        """
        res = np.ndarray(shape=self.shape, dtype=np.float32)
        shift = 0
        for bar_idx in range(-self.bars_count + 1, 1):
            ofs = self._offset + bar_idx
            res[shift] = self._prices.high[ofs]
            shift += 1
            res[shift] = self._prices.low[ofs]
            shift += 1
            res[shift] = self._prices.close[ofs]
            shift += 1
            if self.volumes:
                res[shift] = self._prices.volume[ofs]
                shift += 1
        res[shift] = float(self.have_position)
        shift += 1
        if not self.have_position:
            res[shift] = 0.0
        else:
            res[shift] = self._cur_close() / self.open_price - 1.0
        return res

    def _cur_close(self):
        """
        Calculate real close price for the current bar
        """
        open = self._prices.open[self._offset]
        rel_close = self._prices.close[self._offset]
        return open * (1.0 + rel_close)

    def step(self, action):
        """
        Perform one step in our price, adjust offset, check for the end of prices
        and handle position change
        :param action:
        :return: reward, done
        """
        assert isinstance(action, Actions)
        reward = 0.0
        done = False
        close = self._cur_close()
        if action == Actions.Buy and not self.have_position:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc
        elif action == Actions.Close and self.have_position:
            reward -= self.commission_perc
            done |= self.reset_on_close
            if self.reward_on_close:
                reward += 100.0 * (close / self.open_price - 1.0)
            self.have_position = False
            self.open_price = 0.0

        self._offset += 1
        prev_close = close
        close = self._cur_close()
        done |= self._offset >= self._prices.close.shape[0] - 1

        if self.have_position and not self.reward_on_close:
            reward += 100.0 * (close / prev_close - 1.0)

        return reward, done

In [ ]:
class State1D(State):
    """
    State with shape suitable for 1D convolution
    """

    @property
    def shape(self):
        if self.volumes:
            return (6, self.bars_count)
        else:
            return (5, self.bars_count)

    def encode(self):
        res = np.zeros(shape=self.shape, dtype=np.float32)
        start = self._offset - (self.bars_count - 1)
        stop = self._offset + 1
        res[0] = self._prices.high[start:stop]
        res[1] = self._prices.low[start:stop]
        res[2] = self._prices.close[start:stop]
        if self.volumes:
            res[3] = self._prices.volume[start:stop]
            dst = 4
        else:
            dst = 3
        if self.have_position:
            res[dst] = 1.0
            res[dst + 1] = self._cur_close() / self.open_price - 1.0
        return res

# Trading env

In [ ]:
class StocksEnv(gym.Env):
    metadata = {"render.modes": ["human"]}
    spec = EnvSpec("StocksEnv-v0")

    def __init__(
        self,
        prices,
        bars_count=DEFAULT_BARS_COUNT,
        commission=DEFAULT_COMMISSION_PERC,
        reset_on_close=True,
        state_1d=False,
        random_ofs_on_reset=True,
        reward_on_close=False,
        volumes=False,
    ):
        assert isinstance(prices, dict)
        self._prices = prices
        if state_1d:
            self._state = State1D(
                bars_count,
                commission,
                reset_on_close,
                reward_on_close=reward_on_close,
                volumes=volumes,
            )
        else:
            self._state = State(
                bars_count,
                commission,
                reset_on_close,
                reward_on_close=reward_on_close,
                volumes=volumes,
            )
        self.action_space = gym.spaces.Discrete(n=len(Actions))
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=self._state.shape, dtype=np.float32
        )
        self.random_ofs_on_reset = random_ofs_on_reset
        self.seed()

    def reset(self):
        # make selection of the instrument and it's offset. Then reset the state
        self._instrument = self.np_random.choice(list(self._prices.keys()))
        prices = self._prices[self._instrument]
        bars = self._state.bars_count
        if self.random_ofs_on_reset:
            offset = self.np_random.choice(prices.high.shape[0] - bars * 10) + bars
        else:
            offset = bars
        self._state.reset(prices, offset)
        return self._state.encode()

    def step(self, action_idx):
        action = Actions(action_idx)
        reward, done = self._state.step(action)
        obs = self._state.encode()
        info = {"instrument": self._instrument, "offset": self._state._offset}
        return obs, reward, done, info

    def render(self, mode="human", close=False):
        pass

    def close(self):
        pass

    def seed(self, seed=None):
        self.np_random, seed1 = seeding.np_random(seed)
        seed2 = seeding.hash_seed(seed1 + 1) % 2**31
        return [seed1, seed2]

    @classmethod
    def from_dir(cls, data_dir, **kwargs):
        prices = {file: data.load_relative(file) for file in data.price_files(data_dir)}
        return StocksEnv(prices, **kwargs)

# Model

We will use here the pre-built d3rlpy algos, Here we will use the DQN function

In [ ]:
# training and validation data
data_path = "./YNDX_160101_161231.csv"
val_path = "./YNDX_150101_151231.csv"

stock_data = {"YNDX": load_relative(data_path)}
val_data = {"YNDX": load_relative(val_path)}

Reading ./YNDX_160101_161231.csv
Read done, got 131542 rows, 99752 filtered, 0 open prices adjusted
Reading ./YNDX_150101_151231.csv
Read done, got 130566 rows, 104412 filtered, 0 open prices adjusted


Set up 2 env, one for training and the second for validation.

In [ ]:
# train env
BARS_COUNT = 10
env = StocksEnv(stock_data, bars_count=BARS_COUNT)

In [ ]:
# validation env
env = gym.wrappers.TimeLimit(env, max_episode_steps=1000)
env_val = StocksEnv(val_data, bars_count=BARS_COUNT)

# Online Training

set up a buffer 

In [ ]:
buffer = ReplayBuffer(maxlen= 1000000, env= env)
# create the epsilon-greedy explorer
#explorer = ConstantEpsilonGreedy(0.3)
explorer = LinearDecayEpsilonGreedy(start_epsilon=1.0,
                                    end_epsilon=0.1,
                                    duration=100000)

Init DQN algo and launch training

In [ ]:
# modify weight decay
optim_factory = OptimizerFactory(Adam, weight_decay=1e-4)

In [ ]:
dqn = DQN(
    batch_size=32,
    learning_rate=2.5e-4,
    target_update_interval=100,
    #optim_factory= optim_factory
)

In [ ]:
(env.observation_space.shape)

(32,)

In [ ]:
dqn.build_with_env(env)

In [ ]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment

In [ ]:
evaluate_scorer = evaluate_on_environment(env)

In [ ]:
dqn.fit_online(
    env,
    buffer,
    explorer,
    n_steps=100000,  # train for 100K steps
    eval_env= env_val,
    n_steps_per_epoch=100,  # evaluation is performed every 100 steps
    update_start_step=100,  # parameter update starts after 100 steps
    eval_epsilon=0.3,
    save_metrics= True,
    tensorboard_dir= 'runs'
)

2023-01-10 00:15.34 [info     ] Directory is created at d3rlpy_logs/DQN_online_20230110001534
2023-01-10 00:15.34 [warning  ] Skip building models since they're already built.
2023-01-10 00:15.34 [info     ] Parameters are saved to d3rlpy_logs/DQN_online_20230110001534/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 0.00025, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 100, 'use_gpu': None, 'algorithm': 'DQN', 'observation_shape': (32,), 'action_size': 3}


  0%|          | 0/100000 [00:00<?, ?it/s]

2023-01-10 00:15.34 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_100.pt
2023-01-10 00:15.34 [info     ] DQN_online_20230110001534: epoch=1 step=100 epoch=1 metrics={'time_inference': 0.00025288820266723634, 'time_environment_step': 2.2585391998291017e-05, 'time_step': 0.00029983282089233396, 'rollout_return': -0.31152872342192617, 'evaluation': 0.2991823133979478} step=100
2023-01-10 00:15.34 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_200.pt
2023-01-10 00:15.34 [info     ] DQN_online_20230110001534: epoch=2 step=200 epoch=2 metrics={'time_inference': 0.00026726245880126954, 'time_environment_step': 2.9616355895996093e-05, 'time_sample_batch': 5.3038597106933596e-05, 'time_algorithm_update': 0.001407921314239502, 'loss': 0.01144238221924752, 'time_step': 0.0017974019050598145, 'rollout_return': -0.31639709899078755, 'evaluation': -0.2736961954494065} step=200
2023-01-10 00:15.34 [info     ] Model paramet

2023-01-10 00:15.38 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_1700.pt
2023-01-10 00:15.38 [info     ] DQN_online_20230110001534: epoch=17 step=1700 epoch=17 metrics={'time_inference': 0.0002759885787963867, 'time_environment_step': 3.0465126037597657e-05, 'time_sample_batch': 6.654739379882813e-05, 'time_algorithm_update': 0.001603410243988037, 'loss': 0.015562230846844613, 'time_step': 0.0020165419578552246, 'rollout_return': -0.2119316450908426, 'evaluation': -0.297359922366057} step=1700
2023-01-10 00:15.38 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_1800.pt
2023-01-10 00:15.38 [info     ] DQN_online_20230110001534: epoch=18 step=1800 epoch=18 metrics={'time_inference': 0.0003447437286376953, 'time_environment_step': 4.114389419555664e-05, 'time_sample_batch': 8.385419845581054e-05, 'time_algorithm_update': 0.0018523216247558593, 'loss': 0.018637413321994246, 'time_step': 0.0023844766616821288, 'ro

2023-01-10 00:15.41 [info     ] DQN_online_20230110001534: epoch=32 step=3200 epoch=32 metrics={'time_inference': 0.0002588963508605957, 'time_environment_step': 2.956390380859375e-05, 'time_sample_batch': 6.735801696777343e-05, 'time_algorithm_update': 0.001593327522277832, 'loss': 0.01707541592884809, 'time_step': 0.001987147331237793, 'rollout_return': -0.252488391701142, 'evaluation': -0.3616412690087865} step=3200
2023-01-10 00:15.41 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_3300.pt
2023-01-10 00:15.41 [info     ] DQN_online_20230110001534: epoch=33 step=3300 epoch=33 metrics={'time_inference': 0.0002767658233642578, 'time_environment_step': 3.0925273895263674e-05, 'time_sample_batch': 6.908893585205077e-05, 'time_algorithm_update': 0.0016211938858032226, 'loss': 0.016783953569829466, 'time_step': 0.002034780979156494, 'rollout_return': -0.03699126754431607, 'evaluation': -0.301390354672235} step=3300
2023-01-10 00:15.42 [info     ] Mode

...

2023-01-10 00:19.12 [info     ] DQN_online_20230110001534: epoch=936 step=93600 epoch=936 metrics={'time_inference': 0.00024182796478271483, 'time_environment_step': 2.7632713317871095e-05, 'time_sample_batch': 6.608247756958008e-05, 'time_algorithm_update': 0.0018092894554138184, 'loss': 0.009843543351162226, 'time_step': 0.0021766185760498046, 'rollout_return': -0.672944053936201, 'evaluation': -0.34031074328309846} step=93600
2023-01-10 00:19.12 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_93700.pt
2023-01-10 00:19.12 [info     ] DQN_online_20230110001534: epoch=937 step=93700 epoch=937 metrics={'time_inference': 0.0002411198616027832, 'time_environment_step': 2.8221607208251954e-05, 'time_sample_batch': 6.510019302368164e-05, 'time_algorithm_update': 0.0017986774444580078, 'loss': 0.00900999017059803, 'time_step': 0.002164196968078613, 'rollout_return': -0.022550860743800954, 'evaluation': -0.19381330137360525} step=93700
2023-01-10 00:19.12

2023-01-10 00:19.16 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_95200.pt
2023-01-10 00:19.16 [info     ] DQN_online_20230110001534: epoch=952 step=95200 epoch=952 metrics={'time_inference': 0.00024408578872680664, 'time_environment_step': 2.593517303466797e-05, 'time_sample_batch': 6.554603576660157e-05, 'time_algorithm_update': 0.001856238842010498, 'loss': 0.010127763653872534, 'time_step': 0.00222259521484375, 'rollout_return': -0.2639773096160478, 'evaluation': -0.3965808758011778} step=95200
2023-01-10 00:19.16 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_95300.pt
2023-01-10 00:19.16 [info     ] DQN_online_20230110001534: epoch=953 step=95300 epoch=953 metrics={'time_inference': 0.00025738000869750976, 'time_environment_step': 2.818584442138672e-05, 'time_sample_batch': 6.836652755737305e-05, 'time_algorithm_update': 0.0018764662742614747, 'loss': 0.008590223209466785, 'time_step': 0.002263560295104

2023-01-10 00:19.20 [info     ] DQN_online_20230110001534: epoch=967 step=96700 epoch=967 metrics={'time_inference': 0.0002382969856262207, 'time_environment_step': 2.5413036346435548e-05, 'time_sample_batch': 6.398916244506836e-05, 'time_algorithm_update': 0.0018336606025695801, 'loss': 0.008600681249517947, 'time_step': 0.002191586494445801, 'rollout_return': 0.07691061332776883, 'evaluation': -0.3273033267188584} step=96700
2023-01-10 00:19.20 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_96800.pt
2023-01-10 00:19.20 [info     ] DQN_online_20230110001534: epoch=968 step=96800 epoch=968 metrics={'time_inference': 0.000262603759765625, 'time_environment_step': 3.111839294433594e-05, 'time_sample_batch': 7.407426834106445e-05, 'time_algorithm_update': 0.0019017386436462403, 'loss': 0.008015007502399385, 'time_step': 0.0023038792610168457, 'rollout_return': -2.125643227731094, 'evaluation': 0.6288837389459989} step=96800
2023-01-10 00:19.20 [info 

2023-01-10 00:19.24 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_98300.pt
2023-01-10 00:19.24 [info     ] DQN_online_20230110001534: epoch=983 step=98300 epoch=983 metrics={'time_inference': 0.0002353954315185547, 'time_environment_step': 2.7024745941162108e-05, 'time_sample_batch': 6.507158279418945e-05, 'time_algorithm_update': 0.0017878198623657226, 'loss': 0.01085068276966922, 'time_step': 0.002146754264831543, 'rollout_return': -0.4273753199240392, 'evaluation': -0.22995945940103663} step=98300
2023-01-10 00:19.24 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_98400.pt
2023-01-10 00:19.24 [info     ] DQN_online_20230110001534: epoch=984 step=98400 epoch=984 metrics={'time_inference': 0.0002429962158203125, 'time_environment_step': 2.760171890258789e-05, 'time_sample_batch': 6.615161895751953e-05, 'time_algorithm_update': 0.001822192668914795, 'loss': 0.010744166299700738, 'time_step': 0.002190864086151

2023-01-10 00:19.27 [info     ] DQN_online_20230110001534: epoch=998 step=99800 epoch=998 metrics={'time_inference': 0.0002514195442199707, 'time_environment_step': 2.9206275939941406e-05, 'time_sample_batch': 6.891965866088867e-05, 'time_algorithm_update': 0.0019275355339050293, 'loss': 0.012710023527033628, 'time_step': 0.0023089718818664552, 'rollout_return': 0.20773287414783148, 'evaluation': -0.3968976577701297} step=99800
2023-01-10 00:19.28 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230110001534/model_99900.pt
2023-01-10 00:19.28 [info     ] DQN_online_20230110001534: epoch=999 step=99900 epoch=999 metrics={'time_inference': 0.0002386164665222168, 'time_environment_step': 2.866506576538086e-05, 'time_sample_batch': 6.636381149291993e-05, 'time_algorithm_update': 0.0018741345405578612, 'loss': 0.009352241260930896, 'time_step': 0.0022390007972717283, 'rollout_return': -0.43818730822060004, 'evaluation': 0.3890518766017283} step=99900
2023-01-10 00:19.28 [i

In [ ]:
observations = env.reset()


In [ ]:
scorer = evaluate_on_environment(env)
mean_episode_return = scorer(dqn)
print(mean_episode_return)

0.14403840536286852


# Offline Training

In [ ]:
env = StocksEnv(stock_data, bars_count=BARS_COUNT)

# prepare algorithm
dqn = DQN(
    batch_size=32,
    learning_rate=2.5e-4,
    target_update_interval=100,
)

# prepare replay buffer
buffer = ReplayBuffer(maxlen=1000000, env=env)

# start data collection without updates
dqn.collect(env, buffer)

# export to MDPDataset
dataset = buffer.to_mdp_dataset()

2023-01-10 00:47.17 [debug    ] Building model...
2023-01-10 00:47.17 [debug    ] Model has been built.


  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
# divide dataset into train and test
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

In [ ]:
# initialize neural networks with the given observation shape and action size.
dqn.build_with_dataset(dataset)

2023-01-10 00:50.14 [warning  ] Parameters will be reinitialized.


- td_error_scorer : This metics suggests how Q functions overfit to training sets. If the TD error is large, the Q functions are overfitting.
- average_value_estimation : This metrics suggests the scale for estimation of Q functions. If average value estimation is too large, the Q functions overestimate action-values, which possibly makes training failed.
- evaluate_on_environment : Returns scorer function of evaluation on environment.The metrics of the scorer function is ideal metrics to evaluate the resulted policies.

In [ ]:
dqn.fit(
    train_episodes,
    eval_episodes=test_episodes,
    n_epochs=1000,
    n_steps_per_epoch=1000,
    scorers={
        "td_error": td_error_scorer,
        "value_scale": average_value_estimation_scorer,
        "environment": evaluate_on_environment(env,epsilon= 0.3),
    },
    save_metrics = True,
    tensorboard_dir= 'runs'

)

2023-01-10 00:50.15 [debug    ] RoundIterator is selected.
2023-01-10 00:50.15 [info     ] Directory is created at d3rlpy_logs/DQN_20230110005015
2023-01-10 00:50.15 [warning  ] Skip building models since they're already built.
2023-01-10 00:50.15 [info     ] Parameters are saved to d3rlpy_logs/DQN_20230110005015/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 0.00025, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 100, 'use_gpu': None, 'algorithm': 'DQN', 'observation_shape': (32,), 'action_size': 1}


Epoch 1/1000:   0%|          | 0/24029 [00:00<?, ?it/s]

2023-01-10 00:50.57 [info     ] DQN_20230110005015: epoch=1 step=24029 epoch=1 metrics={'time_sample_batch': 6.626483687519447e-05, 'time_algorithm_update': 0.001359952027453699, 'loss': 2.7710212748940768e-08, 'time_step': 0.0016618959068969235, 'td_error': 1.6867854998729193e-10, 'value_scale': -0.0008007722233549895, 'environment': -0.3433559172330668} step=24029
2023-01-10 00:50.57 [info     ] Model parameters are saved to d3rlpy_logs/DQN_20230110005015/model_24029.pt


Epoch 2/1000:   0%|          | 0/24029 [00:00<?, ?it/s]

2023-01-10 00:51.43 [info     ] DQN_20230110005015: epoch=2 step=48058 epoch=2 metrics={'time_sample_batch': 7.548423100958514e-05, 'time_algorithm_update': 0.0014980420562637458, 'loss': 5.8395315133830114e-09, 'time_step': 0.0018405191877211835, 'td_error': 3.18049013957448e-11, 'value_scale': -2.7675101344597585e-05, 'environment': -0.29663283507369453} step=48058
2023-01-10 00:51.43 [info     ] Model parameters are saved to d3rlpy_logs/DQN_20230110005015/model_48058.pt


...

Epoch 686/1000:   0%|          | 0/24029 [00:00<?, ?it/s]

2023-01-10 11:03.28 [info     ] DQN_20230110005015: epoch=686 step=16483894 epoch=686 metrics={'time_sample_batch': 6.474330981000294e-05, 'time_algorithm_update': 0.001904084861161949, 'loss': 2.4033623060487664e-11, 'time_step': 0.00230990449777678, 'td_error': 2.2097166133866646e-13, 'value_scale': -3.7084588418884316e-05, 'environment': 0.027686299896465705} step=16483894
2023-01-10 11:03.28 [info     ] Model parameters are saved to d3rlpy_logs/DQN_20230110005015/model_16483894.pt


Epoch 687/1000:   0%|          | 0/24029 [00:00<?, ?it/s]

2023-01-10 11:04.25 [info     ] DQN_20230110005015: epoch=687 step=16507923 epoch=687 metrics={'time_sample_batch': 6.740098886808567e-05, 'time_algorithm_update': 0.001888260580814804, 'loss': 2.5109811625988576e-11, 'time_step': 0.0023010940536676194, 'td_error': 5.9149419517186655e-15, 'value_scale': 6.0232386048019344e-06, 'environment': -0.5771945363741058} step=16507923
2023-01-10 11:04.25 [info     ] Model parameters are saved to d3rlpy_logs/DQN_20230110005015/model_16507923.pt


Epoch 688/1000:   0%|          | 0/24029 [00:00<?, ?it/s]